In [59]:
from openai import OpenAI
import settings
import pdfplumber

In [60]:
client = OpenAI(
    api_key = settings.OPENAI_API_KEY
)

In [61]:
def extrair_texto_pdf(caminho_pdf):
    texto_extraido = ""
    with pdfplumber.open(caminho_pdf) as pdf:
        for pagina in pdf.pages:
            texto_extraido += pagina.extract_text() or ""
    return texto_extraido

In [ ]:
caminho_pdf = "docs\DOC1_1002970-50.2022.8.11.0018-1740062052988-4606762-sentenca.pdf"
texto_pdf = extrair_texto_pdf(caminho_pdf)

<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\ggsna\AppData\Local\Temp\ipykernel_22016\2636832216.py:1: SyntaxWarning: invalid escape sequence '\D'
  caminho_pdf = "docs\DOC1_1002970-50.2022.8.11.0018-1740062052988-4606762-sentenca.pdf"


In [85]:
instructions0 = """
Voce é meu assistente de extração de informações de um texto de um PDF.
Eu passarei somente o texto no input, e expero que você me ajude a extrair informações relevantes.
Os documentos sao processos juridicos em que houveram danos socioambientais e preciso montar um banco de dados para fazer a valoracao socioambiental desses danos.

No meu banco de dados, essas serao minhas colunas e uma breve descricao do que eu preciso que esteja nelas:

Número de identificação -> Criar um código identificador do caso para facilitar consulta às informações
Fonte -> Fonte das informações: processo judicial, tese, artigo, manual etc.
Tipo de informação -> Caso real, estudo de caso, simulação etc.
Responsável pelo impacto -> Empresa/Instituição/Pessoa responsável por causar o impacto
Geometria do impacto -> Georreferência
Cidade do impacto -> Cidade do impacto
UF do impacto -> UF do impacto
País do impacto -> País do impacto
Data do impacto -> Data que ocorreu o impacto
Tipo de impacto -> Categoria do dano ambiental (exemplos: desmatamento de APP, derramamento de petróleo, poluição etc.)
Descrição do impacto -> Descrição breve do impacto
Número do processo -> Aplicável para casos extraídos de processos judiciais
Local de tramitação -> Aplicável para casos extraídos de processos judiciais
Status do processo -> Aplicável para casos extraídos de processos judiciais
Meios afetados _> Fauna, Flora, Água superficial, Água subterrânea, Solo etc.
Extensão do impacto (valor) -> Valor numeral 
Extensão do impacto (unidade) -> m2, hectares etc.
Tempo de exposição ao fator de impacto -> Quantificar o tempo de exposição à fonte impactante (tempo de vazamento de petróleo, poluente no ar, efluente, etc)
Contaminante(s) -> Principais contaminantes (petróleo, compostos orgânicos, ar etc.)
Uso do Solo e Cobertura Vegetal -> Principais usos do local impactado antes da ocorrência do impacto
Metodologia valoração -> Metodologia utilizada para valoração do dano
Valor calculado  -> Resultado da valoração na moeda correspondente

Caso voce nao encontre alguma informacao, salve como "NULL".
Nao use separadores de milhar, apenas use "." para separar casas decimais.
Se voce nao tiver mais de 70% de certeza de sua resposta, salve como "NULL".

Exemplo de resposta:

123
Jusbrasil
Processo
Empresa X
-15.000000, -47.000000
São Paulo
SP
Brasil
01/01/2022
Desmatamento de APP
Desmatamento de 10 hectares de APP
1000000000.00
2ª VARA CÍVEL DE JUARA
Julgado
Fauna, Flora
100.00
hectares
1.5 anos
Petróleo
Pastagem
Metodologia X
1000000.00

Responda de modo simples para eu ler com python.

""" 

In [91]:
instructions = """
Você é meu assistente especializado em extrair informações de textos extraídos de PDFs, em especial de processos jurídicos envolvendo danos socioambientais. Sua tarefa é identificar e organizar as informações relevantes contidas no texto e gerar uma resposta estruturada que corresponda ao formato do banco de dados a seguir.

Cada campo da resposta deve seguir rigorosamente as regras abaixo. Se alguma informação não puder ser identificada com pelo menos 70% de certeza, preencha o campo com "NULL".

Formato do banco de dados (uma linha por caso, com cada coluna separada por quebras de linha):

1. Número de identificação: Gere um código único para o caso.
2. Fonte: Indique a fonte da informação (ex.: processo judicial, tese, artigo, manual etc.).
3. Tipo de informação: Informe se é um caso real, estudo de caso, simulação, etc.
4. Responsável pelo impacto: Nome da empresa, instituição ou pessoa responsável pelo dano.
5. Geometria do impacto: Coordenadas geográficas no formato "latitude, longitude" (por exemplo, "-15.000000, -47.000000"). Se não houver, informe "NULL".
6. Cidade do impacto: Nome da cidade onde ocorreu o impacto.
7. UF do impacto: Sigla da Unidade Federativa.
8. País do impacto: Nome do país.
9. Data do impacto: Data em formato DD/MM/AAAA.
10. Tipo de impacto: Categoria do dano ambiental (ex.: desmatamento de APP, derramamento de petróleo, poluição etc.).
11. Descrição do impacto: Breve descrição do impacto.
12. Número do processo: Número do processo judicial, se aplicável.
13. Local de tramitação: Jurisdição ou vara responsável, se aplicável.
14. Status do processo: Status do processo (ex.: Julgado, Em andamento), se aplicável.
15. Meios afetados: Liste os meios impactados (ex.: Fauna, Flora, Água superficial, Água subterrânea, Solo etc.), separados por vírgula.
16. Extensão do impacto (valor): Valor numérico que represente a extensão do impacto. Use "." como separador decimal e não utilize separadores de milhar.
17. Extensão do impacto (unidade): Unidade da extensão (ex.: m2, hectares etc.).
18. Tempo de exposição ao fator de impacto: Tempo de exposição à fonte causadora (ex.: "1.5 anos" ou "36 meses").
19. Contaminante(s): Principais contaminantes (ex.: Petróleo, compostos orgânicos, partículas etc.).
20. Uso do Solo e Cobertura Vegetal: Principais usos do local antes do impacto (ex.: Pastagem, agricultura, floresta).
21. Metodologia valoração: Metodologia utilizada para a valoração do dano.
22. Valor calculado: Valor numérico resultante da valoração, seguindo as mesmas regras numéricas.

Regras gerais:
- Caso alguma informação não seja encontrada com certeza de pelo menos 70%, utilize "NULL" no respectivo campo.
- A resposta deve ser simples e formatada para leitura e processamento em Python, com cada campo em uma nova linha (como no exemplo abaixo).
- Evite adicionar textos explicativos na resposta; retorne somente os valores na ordem correta.

Exemplo de resposta (cada linha representa um campo, na ordem apresentada acima):

123  
Jusbrasil  
Processo  
Empresa X  
-15.000000, -47.000000  
São Paulo  
SP  
Brasil  
01/01/2022  
Desmatamento de APP  
Desmatamento de 10 hectares de APP  
1000000000.00  
2ª VARA CÍVEL DE JUARA  
Julgado  
Fauna, Flora  
100.00  
hectares  
1.5 anos  
Petróleo  
Pastagem  
Metodologia X  
1000000.00
"""


In [92]:
response = client.responses.create(
    model="gpt-4o-mini",
    instructions=instructions,
    input=texto_pdf,
)

print(response.output_text)

1002970-50.2022.8.11.0018  
Processo Judicial Eletrônico  
Processo  
Lucas Zarur Bernardo  
-8.849141, -57.061754  
Juara  
MT  
Brasil  
01/02/2021  
Desmatamento  
Desmatamento ilegal de 207,22 hectares de vegetação nativa em área de especial preservação.  
1002970-50.2022.8.11.0018  
2ª VARA CÍVEL DE JUARA  
Julgado  
Flora  
207.22  
hectares  
1.5 anos  
NULL  
Agricultura, Floresta  
Metodologia de Valoração de Danos Ambientais  
1071563.63  


In [93]:
texto_extraido = response.output_text

In [94]:
texto_extraido = texto_extraido.split("\n")

In [97]:
texto_extraido = [s.rstrip() for s in texto_extraido]

In [98]:
texto_extraido

['1002970-50.2022.8.11.0018',
 'Processo Judicial Eletrônico',
 'Processo',
 'Lucas Zarur Bernardo',
 '-8.849141, -57.061754',
 'Juara',
 'MT',
 'Brasil',
 '01/02/2021',
 'Desmatamento',
 'Desmatamento ilegal de 207,22 hectares de vegetação nativa em área de especial preservação.',
 '1002970-50.2022.8.11.0018',
 '2ª VARA CÍVEL DE JUARA',
 'Julgado',
 'Flora',
 '207.22',
 'hectares',
 '1.5 anos',
 'NULL',
 'Agricultura, Floresta',
 'Metodologia de Valoração de Danos Ambientais',
 '1071563.63']